In [1]:
# Data sorted into pandas dataframe and extractions

In [2]:
import os, sys, yaml, re
from IPython.display import display, Markdown, Latex #Can write latex too!!!!
import numpy as np

In [3]:
class DecayChain(yaml.YAMLObject):
    
    yaml_tag = u'!DecayChain'
    
    def __init__(self, idd="", be="", pi="", ie="", it="", ea=[], eas=[], ta=[]):
        self.ID = idd
        self.BeamEnergy = be
        self.Pixel = pi
        self.ImplantEnergy = ie
        self.ImplantTime = it
        self.EAlpha, self.EAlphaSigma, self.TAlpha = ea, eas, ta

    def __repr__(self):
        return yaml.dump(self)
    
    def __str__(self):
        return yaml.dump(self)
    
    def MDTable(self):
        #Data to insert
        rows = []
        rows.append(['**ID**', '$E_{lab}$ (MeV)', '$E_{rec}$ (MeV)']) #'$E_1$ (MeV)', '$E_2$ (MeV)']
        rows.append(['', '', '**pixel**']) #'**$\Delta t_2$ (s)**']
        rows.append([self.ID, self.BeamEnergy, self.ImplantEnergy])
        rows.append(['', '', self.Pixel])
        for i in range(len(self.EAlpha)):
            rows[0].append('$E_'+str(i)+'$ (MeV)')
            rows[1].append('$\Delta t_'+str(i)+'$ (s)')
            rows[2].append(self.EAlpha[i])
            rows[3].append(self.TAlpha[i])
        rows[2] = [str(i) for i in rows[2]]
        rows[3] = [str(i) for i in rows[3]]
        
        #Formatting strings where data is to be put in
        s_format = """| """
        s_hline = """|"""
        s_empty = """| """

        for i in range(len(rows[0])):
            s_format += """{"""+str(i)+"""} | """
            s_hline += """:---:| """
            s_empty += """ | """
            if i == len(rows[0])-1:
                s_format += """\n"""
                s_hline += """\n"""
                s_empty += """\n"""

        s_md = """"""
        s_md += s_empty+s_hline
        for r in rows: 
            s_md += s_format.format(*r)
        
        #print("string:\n", s_md)
        #print(Markdown(s_md))
        display(Markdown(s_md))
        return s_md

class SetDecayChains:
    
    def __init__(self, path='', ids=[]):
        self.max_chain = 0
        s_files = " ".join(os.listdir(path))
        files = []
        for s in ids:
            files += (sorted(re.findall(string=s_files, pattern="Chain"+s+"\d+.yml")))
        print("Reading the following files:", files)
        self.Chains = []
        for f in files:
            f_in = open(path+f, 'r')
            self.Chains.append(yaml.load(f_in))
            if len(self.Chains[-1].TAlpha) > self.max_chain:
                self.max_chain = len(self.Chains[-1].TAlpha)
            f_in.close()
   

In [4]:
s_path = "data/"
s_id = ["13", "14", "11", "12", "16", "17", "20"]

setDC = SetDecayChains(s_path, s_id)
#for c in setDC.Chains:
#    c.MDTable()
print("Max length=", setDC.max_chain)

Reading the following files: ['Chain1301.yml', 'Chain1302.yml', 'Chain1303.yml', 'Chain1304.yml', 'Chain1305.yml', 'Chain1306.yml', 'Chain1307.yml', 'Chain1308.yml', 'Chain1309.yml', 'Chain1310.yml', 'Chain1311.yml', 'Chain1312.yml', 'Chain1313.yml', 'Chain1314.yml', 'Chain1315.yml', 'Chain1316.yml', 'Chain1317.yml', 'Chain1318.yml', 'Chain1319.yml', 'Chain1320.yml', 'Chain1321.yml', 'Chain1322.yml', 'Chain1401.yml', 'Chain1402.yml', 'Chain1403.yml', 'Chain1404.yml', 'Chain1405.yml', 'Chain1406.yml', 'Chain1407.yml', 'Chain1101.yml', 'Chain1102.yml', 'Chain1103.yml', 'Chain1104.yml', 'Chain1201.yml', 'Chain1202.yml', 'Chain1601.yml', 'Chain1602.yml', 'Chain1603.yml', 'Chain1604.yml', 'Chain1605.yml', 'Chain1606.yml', 'Chain1607.yml', 'Chain1608.yml', 'Chain1609.yml', 'Chain1610.yml', 'Chain1611.yml', 'Chain1612.yml', 'Chain1613.yml', 'Chain1614.yml', 'Chain1615.yml', 'Chain1616.yml', 'Chain1617.yml', 'Chain1618.yml', 'Chain1619.yml', 'Chain1620.yml', 'Chain1621.yml', 'Chain1622.yml', '

In [5]:
import pandas as pd

## One chain in a data frame

* Indices = chain identification
* Series = values of variables, Energy, Lifetime ...
* Data Frame = everything together

In [6]:
c1 = setDC.Chains[0]
c1

!DecayChain
BeamEnergy: 242.1
EAlpha: [10.29, 9.24, 9.77, 0.825, 9.06, 197.0]
EAlphaSigma: [10.0, 10.0, 10.0, 30.0, 10.0, 0.0]
ID: '1301'
ImplantEnergy: 13.4
ImplantTime: 0.0
Pixel: 465
TAlpha: [0.366, 0.935, 11.1, 0.515, 15.3, 17.4]

In [7]:
inds = [ [c1.ID, '12'], ["Dubna", '13'], ["E115", '14'] ]
col_names = ["ID", "Lab", "Exp."]
inds = list(zip(*inds))
print(inds)
index = pd.MultiIndex.from_tuples(inds, names=col_names)
ser_it = {"Implantation time": c1.ImplantTime}
index

[('1301', 'Dubna', 'E115'), ('12', '13', '14')]


MultiIndex(levels=[['12', '1301'], ['13', 'Dubna'], ['14', 'E115']],
           labels=[[1, 0], [1, 0], [1, 0]],
           names=['ID', 'Lab', 'Exp.'])

In [8]:
pd.Series(data=ser_it, index=index)

ID    Lab    Exp.
1301  Dubna  E115   NaN
12    13     14     NaN
dtype: float64

In [9]:
pd.DataFrame(data=ser_it, index=index)

,,,Implantation time
ID,Lab,Exp.,
1301,Dubna,E115,0.0
12,13,14,0.0


## Generic data frame implementation

Filling arrays

In [10]:
ind_names = ["Element", "Type", "Lab", "ID"]
#inds = [ [c1.ID], ["Dubna"], ["E115"] ]
index = np.empty((len(setDC.Chains), len(ind_names)), dtype="<U20")
col_names = ["Beam Energy (MeV)", "Implantation Energy (MeV)", "Implantation time (s)", "Pixel"]
singles = np.empty((len(setDC.Chains), len(col_names)))

tot_nuclei = setDC.max_chain+1 #number of nuclei = columns per energy and time
rest = np.full((len(setDC.Chains), 3*tot_nuclei), np.nan)
E_names = ["E117", "E115", "E113", "Rg", "Mt", "Bh", "Db", "Lr"]
rest_str = ["Energy {} (MeV)", "$\sigma_E$ {} (keV)", "Lifetime {} (s)"]
rest_names = []
for i in range(3):
    for j in range(tot_nuclei):
        rest_names.append(rest_str[i].format(E_names[j]))
print("Rest_names=", rest_names)

E117Link = ['03', '04', '05', '06', '09', '10', '11', '12', '13', '14']
offset = 1 #This is for E117 or E115 insertion with shift
for i, c in enumerate(setDC.Chains):
    idi = c.ID[0:2]
    if idi == '10' or idi == '11' or idi == '12':
        offset = 1
        lab = "Dubna"
        if idi == "10":
            chain, ttype = 'E115', '3n'
        elif idi == '12':
            chain, ttype = 'E115', '4n'
        else:
            chain, ttype = 'E115', 'Short'
    if idi == '13' or idi == '14' or idi == '15':
        offset = 1
        lab = "GSI"
        if idi == "13":
            chain, ttype = 'E115', '3n'
        elif idi =='15':
            chain, ttype = 'E115', '4n'
        else:
            chain, ttype = 'E115', 'Short'
    if idi == '16' or idi == '17':
        offset = 1
        lab = "Berkeley"
        if idi == "16":
            chain, ttype = 'E115', '3n'
        else:
            chain, ttype = 'E115', 'Short'
    if idi == '20':
        offset = 0
        lab = "Dubna"
        chain = 'E117'
        if c.ID[2:] in E117Link:
            ttype = 'Link'
        else:
            ttype = '-'
    index[i] = np.asarray([chain, ttype, lab, str(int(c.ID[2:]))])
    singles[i] = np.asarray([c.BeamEnergy, c.ImplantEnergy, c.ImplantTime, c.Pixel])
    rest[i, offset:len(c.EAlpha)+offset] = c.EAlpha
    rest[i, offset+tot_nuclei:tot_nuclei+len(c.EAlpha)+offset] = c.EAlphaSigma
    rest[i, offset+2*tot_nuclei:2*tot_nuclei+len(c.EAlpha)+offset] = c.TAlpha
#index = list(zip(*index.T))
#print(index)
index = pd.MultiIndex.from_arrays(index.T, names=ind_names)
index

Rest_names= ['Energy E117 (MeV)', 'Energy E115 (MeV)', 'Energy E113 (MeV)', 'Energy Rg (MeV)', 'Energy Mt (MeV)', 'Energy Bh (MeV)', 'Energy Db (MeV)', 'Energy Lr (MeV)', '$\\sigma_E$ E117 (keV)', '$\\sigma_E$ E115 (keV)', '$\\sigma_E$ E113 (keV)', '$\\sigma_E$ Rg (keV)', '$\\sigma_E$ Mt (keV)', '$\\sigma_E$ Bh (keV)', '$\\sigma_E$ Db (keV)', '$\\sigma_E$ Lr (keV)', 'Lifetime E117 (s)', 'Lifetime E115 (s)', 'Lifetime E113 (s)', 'Lifetime Rg (s)', 'Lifetime Mt (s)', 'Lifetime Bh (s)', 'Lifetime Db (s)', 'Lifetime Lr (s)']


MultiIndex(levels=[['E115', 'E117'], ['-', '3n', '4n', 'Link', 'Short'], ['Berkeley', 'Dubna', 'GSI'], ['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '5', '6', '7', '8', '9']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 0, 0, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [11]:
print(col_names + rest_names)
data = np.hstack((singles, rest))

['Beam Energy (MeV)', 'Implantation Energy (MeV)', 'Implantation time (s)', 'Pixel', 'Energy E117 (MeV)', 'Energy E115 (MeV)', 'Energy E113 (MeV)', 'Energy Rg (MeV)', 'Energy Mt (MeV)', 'Energy Bh (MeV)', 'Energy Db (MeV)', 'Energy Lr (MeV)', '$\\sigma_E$ E117 (keV)', '$\\sigma_E$ E115 (keV)', '$\\sigma_E$ E113 (keV)', '$\\sigma_E$ Rg (keV)', '$\\sigma_E$ Mt (keV)', '$\\sigma_E$ Bh (keV)', '$\\sigma_E$ Db (keV)', '$\\sigma_E$ Lr (keV)', 'Lifetime E117 (s)', 'Lifetime E115 (s)', 'Lifetime E113 (s)', 'Lifetime Rg (s)', 'Lifetime Mt (s)', 'Lifetime Bh (s)', 'Lifetime Db (s)', 'Lifetime Lr (s)']


In [12]:
data = dict(zip(col_names+rest_names, data.T))
data

{'$\\sigma_E$ Bh (keV)': array([  10.    ,   10.    ,   10.    ,   40.    ,   10.    ,   10.    ,
          50.    ,    0.    ,    0.    ,   50.    ,   10.    ,   10.    ,
          10.    ,   40.    ,   10.    ,   10.    ,    0.    ,   40.    ,
          40.    ,   50.    ,    0.    ,   10.    ,       nan,       nan,
              nan,       nan,       nan,       nan,       nan,       nan,
              nan,       nan,       nan,   68.7898,    0.    ,   20.    ,
          50.    ,   20.    ,   20.    ,    0.    ,   20.    ,   20.    ,
         100.    ,   50.    ,   20.    ,   20.    ,   20.    ,    0.    ,
          20.    ,   20.    ,  200.    ,   20.    ,   20.    ,   20.    ,
          20.    ,   20.    ,   50.    ,   20.    ,    0.    ,    0.    ,
           0.    ,    0.    ,   20.    ,   20.    ,   20.    ,   50.    ,
          50.    ,   20.    ,   50.    ,   50.    ,   50.    ,   20.    ,
          50.    ,   20.    ,   50.    ,   20.    ,   50.    ,    0.    ,
              

In [13]:
df = pd.DataFrame(data=data, index=index, columns=col_names+rest_names)
df

Beam Energy (MeV)  Implantation Energy (MeV)  \
Element Type  Lab      ID                                                 
E115    3n    GSI      1               242.1                      13.40   
                       2               242.1                      13.90   
                       3               242.1                      16.30   
                       4               242.1                      16.10   
                       5               242.1                      16.40   
                       6               242.1                      15.40   
                       7               242.1                      14.10   
                       8               242.1                      12.50   
                       9               245.0                      15.90   
                       10              245.0                      14.50   
                       11              245.0                      16.50   
                       12              245.0                      15.10   
                       13              245.0                      15.30   
                       14              245.0                      14.90   
                       15              245.0                      17.20   
                       16              245.0                      12.60   
                       17              245.0                      16.30   
                       18              245.0                      14.40   
                       19              245.0                      14.90   
                       20              245.0                      15.00   
                       21              245.0                      15.10   
                       22              245.0                      14.50   
        Short GSI      1               245.0                      12.30   
                       2               242.1                      16.20   
                       3               242.1                      13.90   
                       4               242.1                      14.50   
                       5               242.1                      13.80   
                       6               245.0                      14.50   
                       7               245.0                      11.90   
              Dubna    1               240.5                      11.38   
...                                      ...                        ...   
        3n    Berkeley 33                0.0                      13.59   
                       34                0.0                       0.00   
                       35                0.0                      10.19   
                       36                0.0                      13.79   
                       37                0.0                      11.93   
                       38                0.0                      11.64   
                       39                0.0                      14.55   
                       40                0.0                      13.71   
                       41                0.0                      12.30   
                       42                0.0                      15.16   
                       43                0.0                      12.10   
        Short Berkeley 1               270.0                      11.65   
                       2               270.0                      11.18   
                       3               270.0                      13.72   
E117    -     Dubna    1                 0.0                      11.89   
                       2               252.0                      13.87   
        Link  Dubna    3               245.0                      13.51   
                       4               245.0                       9.96   
                       5               245.0                       9.36   
                       6                 0.0                       9.91   
        -     Dubna    7                 0.0        

In [14]:
df["$\sigma_E$ E117 (keV)"]

Element  Type   Lab       ID
E115     3n     GSI       1       NaN
                          2       NaN
                          3       NaN
                          4       NaN
                          5       NaN
                          6       NaN
                          7       NaN
                          8       NaN
                          9       NaN
                          10      NaN
                          11      NaN
                          12      NaN
                          13      NaN
                          14      NaN
                          15      NaN
                          16      NaN
                          17      NaN
                          18      NaN
                          19      NaN
                          20      NaN
                          21      NaN
                          22      NaN
         Short  GSI       1       NaN
                          2       NaN
                          3       NaN
                     

## Pickle to file

In [15]:
df.to_pickle('data/ChainsDataFrame.p')

### Read from pickled file

In [16]:
df = pd.read_pickle('data/ChainsDataFrame.p')
df

Beam Energy (MeV)  Implantation Energy (MeV)  \
Element Type  Lab      ID                                                 
E115    3n    GSI      1               242.1                      13.40   
                       2               242.1                      13.90   
                       3               242.1                      16.30   
                       4               242.1                      16.10   
                       5               242.1                      16.40   
                       6               242.1                      15.40   
                       7               242.1                      14.10   
                       8               242.1                      12.50   
                       9               245.0                      15.90   
                       10              245.0                      14.50   
                       11              245.0                      16.50   
                       12              245.0                      15.10   
                       13              245.0                      15.30   
                       14              245.0                      14.90   
                       15              245.0                      17.20   
                       16              245.0                      12.60   
                       17              245.0                      16.30   
                       18              245.0                      14.40   
                       19              245.0                      14.90   
                       20              245.0                      15.00   
                       21              245.0                      15.10   
                       22              245.0                      14.50   
        Short GSI      1               245.0                      12.30   
                       2               242.1                      16.20   
                       3               242.1                      13.90   
                       4               242.1                      14.50   
                       5               242.1                      13.80   
                       6               245.0                      14.50   
                       7               245.0                      11.90   
              Dubna    1               240.5                      11.38   
...                                      ...                        ...   
        3n    Berkeley 33                0.0                      13.59   
                       34                0.0                       0.00   
                       35                0.0                      10.19   
                       36                0.0                      13.79   
                       37                0.0                      11.93   
                       38                0.0                      11.64   
                       39                0.0                      14.55   
                       40                0.0                      13.71   
                       41                0.0                      12.30   
                       42                0.0                      15.16   
                       43                0.0                      12.10   
        Short Berkeley 1               270.0                      11.65   
                       2               270.0                      11.18   
                       3               270.0                      13.72   
E117    -     Dubna    1                 0.0                      11.89   
                       2               252.0                      13.87   
        Link  Dubna    3               245.0                      13.51   
                       4               245.0                       9.96   
                       5               245.0                       9.36   
                       6                 0.0                       9.91   
        -     Dubna    7                 0.0        